# Kapitel 3 - Erste Textklassifikation mit Naive Bayes 

## 3.1. Kapitelübersicht <a class="anchor" id="3-1"/>

In diesem Kapitel lernen wir unserer erstes Textklassifikationsverfahren kennen, das <b>Naive Bayes</b>-Verfahren. Es basiert auf dem Bayesschen Theorem, das von Thomas Bayes schon Mitte des 18. Jahrhunderts publiziert wurde. Das Naive Bayes Verfahren selbst ist seit den 1950er und 1960er ein intensiv untersuchtes Verfahren und wird selbst heute noch aufgrund seiner Simplizität als Textklassifizierungsverfahren genutzt.<br>
Es eignet sich hervorragend als "Einsteiger"-Verfahren, da man bei einem kleinen Datensatz die Klassifizierungsmethode selbst händisch und ohne Programmierwerkzeuge durchführen kann, was wir in Abschnitt 3.3. auch tun werden. Zuletzt werden wir in diesem Kapitel erstmals einen Einblick in die Software-Bibliothek für Maschinelles Lernen in Python <b>Scikit-learn</b> erhalten und mithilfe von existierenden Modulen das Naive Bayes Verfahren implementieren und auf den Beispieldatensatz anwenden.

<b>Abschnittsübersicht</b><br>

[3.1. Kapitelübersicht](#3-1)<br>
[3.2. Das Naive Bayes Verfahren](#3-2)<br>
[3.3. Eine erste, manuell durchgeführte Anwendung](#3-3)<br>
[3.4. Naive Bayes mit Scikit learn](#3-4)<br>
[3.5. Mögliche Fehler](#3-5)<br>


Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- Naive Bayes, speziell das Multinomial Naive Bayes
- Bayes-Theorem/ Satz von Bayes
- Simples Feature Engineering mit dem Bag-of-Words Modell
- A-priori-Wahrscheinlichkeit
- Erste Schritte in Scikit learn
- Sparse und Dense Matrizen

## 3.2. Das Naive Bayes Verfahren <a class="anchor" id="3-2"/>

Das <b>Naive Bayes</b> Verfahren ist ein Klassifikationsverfahren, welches auf dem <b>Bayes-Theorem</b> (auch <b>"Satz von Bayes"</b> genannt) von Thomas Bayes basiert. Beim Naive Bayes Verfahren wird <u>naiv</u> (deshalb auch der Name) angenommen, dass alle Features der Eingabedaten (= input data) <b>unabhängig</b> voneinander sind. Obwohl das in der Realität selten der Fall ist, liefert Naive Bayes in der Praxis meist gute Werte.<br>
Das Naive Bayes Verfahren besteht nicht nur aus einem einzigen Algorithmus, sondern vielmehr aus einer Familie von Algorithmen, die alle auf dem gleichen Prinzip beruhen. Alle Algorithmen basieren zudem auf der <b>probabilistischen Methode</b>. Dies bedeutet, dass Naive Bayes beispielsweise die Zugehörigkeit einer Kategorie zu einem Text für jede Kategorie berechnet und die höchste Wahrscheinlichkeit als richtige Kategorie zurückgibt. Die Wahrscheinlichkeiten werden durch das <b>Bayes-Theorem</b> ermittelt. Die Formel für das Theorem lautet: <br>

$ P(A|B) = \frac{P(B|A) \ \cdot \ P(A)}{P(B)} $ &nbsp; [<sup>1</sup>](#fn1) <br>



<hr style="border: 0.1px solid black;"/>
<span id="fn1" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">1</sup> &nbsp; Zwei anschauliche Rechenbeispiele mit Daten, die nicht aus Text bestehen, findet man auf der Wikipedia Seite vom <a href=https://de.wikipedia.org/wiki/Satz_von_Bayes>Satz von Bayes</a>.</span>

## 3.3. Eine erste, manuell durchgeführte Anwendung <a class="anchor" id="3-3"/>

Um uns mit dem Naive Bayes Verfahren vertraut zu machen, werden wir zunächst ein einfaches Beispiel[<sup>2</sup>](#fn2) durchrechnen, bevor wir das Verfahren auf einen Datensatz anwenden. Dazu werden wir das <b>Multinomial Naive Bayes</b> Verfahren nutzen, eine der vielen Varianten von Naive Bayes. Dieses Verfahren wird genutzt, wenn eine multinomiale Verteilung der Daten vorliegt, es also mehr als zwei mögliche Outputs geben kann. Es funktioniert gut mit Textdaten, bei denen die Worthäufigkeiten berücksichtigt werden sollen. Bevor wir jedoch das Verfahren verwenden, schauen wir uns den Datensatz an. 

<hr style="border: 0.1px solid black;"/>
<span id="fn2" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">2</sup> &nbsp; Dieses Beispiel wurde von dem <a href=https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/>Naive Bayes Tutorial</a> der Plattform <b>MonkeyLearn</b> übernommen, übersetzt und leicht abgewandelt.</span>

In [1]:
import pandas as pd

news = {1 : ["Ein großartiges Spiel", "Sport"], 
        2 : ["Das Urteil ist beschlossen", "Kein Sport"],
        3 : ["Ein sehr faires Match", "Sport"],
        4 : ["Ein faires aber langweiliges Spiel", "Sport"],
        5 : ["Es ist ein knappes Urteil", "Kein Sport"],
        6 : ["Ein sehr knappes Spiel", "?"]}
simple_df = pd.DataFrame.from_dict(news, orient="index", columns=["Text", "Kategorie"])

simple_df

,Text,Kategorie
1,Ein großartiges Spiel,Sport
2,Das Urteil ist beschlossen,Kein Sport
3,Ein sehr faires Match,Sport
4,Ein faires aber langweiliges Spiel,Sport
5,Es ist ein knappes Urteil,Kein Sport
6,Ein sehr knappes Spiel,?


Wir sehen eine Tabelle, in der kurze Nachrichtenartikel in eine von zwei Kategorien eingeteilt wurden: "Sport" oder "Kein Sport". Der sechste Artikel "Ein sehr knappes Spiel" wurde in keine Kategorie eingeteilt. Unsere Aufgabe ist es jetzt, mithilfe des <b>Multinomial Naive Bayes</b> Klassifikationsverfahren die Wahrscheinlichkeiten zu berechnen, ob der Text "Ein sehr knappes Spiel" in die Kategorie "Sport" oder "Kein Sport" eingeteilt wird. Die <u>größere</u> Wahrscheinlichkeit bestimmt, zu welcher Kategorie der Satz gehört.
Mathematisch wird das folgenderweise ausgedrückt: <br>

$ P(Sport|Ein\ sehr\ knappes\ Spiel) $ <br>

$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) $

Wir haben nun ein Problem: Da unsere Daten aus Texten bestehen, können wir nicht mit ihnen rechnen und sie deshalb nicht in die Formel des Bayesschen Theorems einsetzen. Deshalb müssen wir zunächst <b>"Feature Engineering"</b> betreiben, einen zentralen Schritt bei der Nutzung von Machine Learning Algorithmen. 

<div class="alert alert-info">
<b>Exkurs:</b> Features, Feature Selection, Feature Extraction und Feature Engineering

<b>Features</b> (deutsch: Eigenschaften, Merkmale) sind messbare Eigenschaften eines Objekts. Anders als <b>Attribute</b> sollen Features <u>aussagekräftig</u> sein. Unbedeutende Features sollen aussortiert werden. Dieser Vorgang wird <b>Feature Selection </b> genannt. Features sind nicht immer direkt nutzbar, sondern müssen aus den Objekten extrahiert werden. Diesen Vorgang nennt man <b>Feature Extraction</b>.

> Feature extraction is the transformation of original data to a data set with a reduced number of variables, which contains the most discriminatory information.[<sup>3</sup>](#fn3)

Der Feature Extraction sehr ähnlich ist das <b>Feature Engineering</b>. Die beiden Begriffe werden oft synonym verwendet, tatsächlich umfasst das Feature Engineering aber die Vorgänge <b>Feature Extraction</b> und <b>Feature Selection</b>. Feature Engineering beschreibt den Vorgang, die vorhandenen Daten in eine Form zu bringen, die ein maschinelles Lernverfahren nutzen kann. Grundsätzlich verwenden alle Algorithmen für maschinelles Lernen Eingabedaten, um Ausgaben zu erstellen. Diese Eingabedaten umfassen Features, die nicht immer direkt von maschinellen Lernverfahren genutzt werden können und vorverarbeitet werden müssen. Dafür ist das Feature Engineering zuständig.<br>
Feature Engineering ist nicht immer der selbe Vorgang, sondern kann je nach Problemstellung anders aussehen. So ist das Problem bei Textdaten meistens, dass das maschinelle Lernverfahren mit der Repräsentation von Wörtern und Sätzen nichts anfangen kann, da es auf diese Form der Daten keine mathematische Rechenoperationen ausführen kann. Deshalb müssen die Textdaten in numerische Features transformiert werden (<b>Feature Transformation</b>). 

Im Grunde verfolgt Feature Engineering zwei Ziele: 
- Die Vorbereitung des richtigen Eingabedatensatzes, der mit den Anforderungen des maschinellen Lernalgorithmus kompatibel ist.
- Verbesserung der Leistung von Modellen für maschinelles Lernen.

Für weitere Informationen zu Feature Engineering, Feature Extraction und Feature Selection siehe den <a href="https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/">Blog-Eintrag</a> von Jason Brownlee.


<hr style="border: 0.1px solid black;"/>
<span id="fn3" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">3</sup> &nbsp; Siehe: <a href="https://www.sciencedirect.com/topics/engineering/feature-extraction">https://www.sciencedirect.com/topics/engineering/feature-extraction</a> (abgerufen am 03.09.2019).</span>

</div>

Wir müssen uns dazu entschieden, welche Features wir aus den Texten extrahieren wollen. Würde unser Datensatz aus Personen bestehen und wir würden eine Klassifikation mit den Klassen "Gesund" und "Nicht Gesund" durchführen wollen, könnten wir als Features die Körpergröße, das Gewicht oder das Geschlecht nehmen (<b>Feature Extraction</b>) und solche Features wie Vornamen, Lieblingsfarbe oder Sprachkenntnisse könnten wir weglassen (<b>Feature Selection</b>).<br>
In unserem Fall haben wir jedoch keine numerischen Features, sondern nur Text. Um den Text in die Formel des <b>Bayesschen Theorems</b> einsetzen zu können, müssen wir den Text in Zahlen transformieren. Eine Möglichkeit bestünde darin, die einfachen Worthäufigkeiten jedes Textes zu nehmen. Dieses Verfahren nennt sich <b>Bag-of-Words</b>.

<div class="alert alert-info">
<b>Exkurs:</b> Bag-of-Words Modell

Ein <b>Bag-of-Words</b> Modell (kurz: BoW) ist eine Methode, um Features aus einem Text zu extrahieren, damit man diese für eine Modellierung, wie hier für einen Machine Learning-Algorithmus, verwenden kann. Die Idee hierbei ist, dass Dokumente, die eine ähnliche Anzahl der gleichen Wörter aufweisen, einen ähnlichen Inhalt haben.

Der Bag-of-Words-Ansatz ist simpel, flexibel und kann vielseitig verwendet werden. Er beschreibt das Auftreten von Wörtern in einem Dokument und besteht aus einem Vokabular der bekannten Wörter und einem Maß für das Vorkommen der bekannten Wörter. Die Bezeichnung "bag" (deutsch: <i>Sack</i>) soll darauf hinweisen, dass alle Informationen über die Struktur oder Reihenfolge der Wörter im Dokument verworfen werden. Es ist also nicht mehr möglich, die Lage eines Wortes nach der Umwandlung eines Textes in einem Bag-of-words zu bestimmen.

Das Bag-of-Words-Modell kann erweitert werden, etwa durch <b>tf-idf</b> oder <b>N-Gramme</b> (n-grams).
</div>

Bevor wir aber unseren Text in ein <b>Bag-of-words</b>-Modell überführen, setzen wir zunächst den Satz "Ein sehr knappes Spiel" in die Formel für das <b>Bayes-Theorem</b> ein. Wir erhalten somit:

$ P(Sport|Ein\ sehr\ knappes\ Spiel) = \frac{P(Ein\ sehr\ knappes\ Spiel|Sport) \ \cdot \ P(Sport)}{P(Ein\ sehr\ knappes\ Spiel)}  $ <br>


$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) = \frac{P(Ein\ sehr\ knappes\ Spiel|Kein\ Sport) \ \cdot \ P(Kein\ Sport)}{P(Ein\ sehr\ knappes\ Spiel)}  $ <br>

Da wir bei unserem Vergleich nur herausfinden wollen, welche Kategorie die höhere Wahrscheinlichkeit hat, können wir zur Vereinfachung den Divisor weglassen und erhalten somit die folgenden gekürzten Formeln:

$ P(Sport|Ein\ sehr\ knappes\ Spiel) = P(Ein\ sehr\ knappes\ Spiel|Sport) \ \cdot \ P(Sport)  $ <br>


$ P(Kein\ Sport|Ein\ sehr\ knappes\ Spiel) = P(Ein\ sehr\ knappes\ Spiel|Kein\ Sport) \ \cdot \ P(Kein\ Sport)  $ <br>


Jetzt müssten wir eigentlich nur noch zählen, wie oft der Satz "Ein sehr knappes Spiel" in der Kategorie "Sport" oder "Kein Sport" auftritt und das durch die Gesamtanzahl aller Texte teilen. Somit würden wir $ P(Ein\ sehr\ knappes\ Spiel|Sport) $ oder $ P(Ein\ sehr\ knappes\ Spiel|Kein\ Sport) $ berechnen können. Das Problem ist jedoch, dass dieser Satz in den anderen Texten gar nicht vorkommt und die Wahrscheinlichkeit für beide Kategorien $ P = 0 $ wäre. Unser Modell wäre nur sinnvoll, wenn wir sehr große Texte hätten und die Texte den exakten Satz beinhalten würden.<br>

Wir müssen uns deshalb den <i>naiven</i> Teil vom <b>Naive Bayes</b> Verfahren zu Nutze machen. Anstatt dass wir den gesamten Satz betrachten, betrachten wir die einzelnen Wörter <u>unabhängig</u> voneinander. Sätze wie "ich gehe nicht nach hause" und "nicht nach hause gehe ich" werden nicht unterschieden, für unser Modell sind es die gleichen Sätze, da sie die gleichen Wörter beinhalten.<br>

Mithilfe dieser Annahme können wir nun die Wörter in $ P(Ein\ sehr\ knappes\ Spiel) $ einsetzen und erhalten:<br>

$ P(Ein\ sehr\ knappes\ Spiel) = P(Ein) \cdot P(sehr) \cdot P(knappes) \cdot P(Spiel) $ <br>

Das setzen wir nun in $ P(Ein\ sehr\ knappes\ Spiel|Sport) $ ein und erhalten somit:<br>

$ P(Ein\ sehr\ knappes\ Spiel|Sport) = P(Ein|Sport) \cdot P(sehr|Sport) \cdot P(knappes|Sport) \cdot P(Spiel|Sport) $ <br>



Jetzt können wir endlich unsere Wahrscheinlichkeiten berechnen, indem wir die Wörter in unseren Daten zusammenzählen. Dafür berechnen wir zunächst für jede der beiden Kategorien ihre <i>A-priori</i>-Wahrscheinlichkeit: Die Wahrscheinlickeit, dass ein Text in unserem Datensatz eine der beiden Kategorien angehört,[<sup>4</sup>](#fn4) ist für $ P(Sport) $ somit $ \frac{3}{5} $ und für $ P(Kein\ Sport) $ ist sie $ \frac{2}{5}$.


<hr style="border: 0.1px solid black;"/>
<span id="fn4" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">4</sup> &nbsp; Ausgenommen wird hier natürlich der Satz "Ein sehr knappes Spiel", der (noch) keiner Kategorie zugewiesen ist und deshalb nicht berücksichtigt wird.</span>

<div class="alert alert-info">
<b>Exkurs:</b> <i>A-priori</i>-Wahrscheinlichkeit<br>

Die A-priori-Wahrscheinlichkeit ist ein <b>Wahrscheinlichkeitswert</b>, der aufgrund von Vorwissen gewonnen werden kann. Möchte man beispielsweise die Wahrscheinlichkeit eines Würfelwurfs berechnen, kann man schon vor dem Wurf sagen, dass die Wahrscheinlichkeit des Auftretens einer bestimmten Zahl $ \frac{1}{6} $ ist, da man vorher wusste, dass der Würfel sechs Seiten hat. Das gleiche gilt für einen Münzwurf, da wir dort wissen, dass eine Münze nur zwei Seiten hat und deshalb die Wahrscheinlichkeit $ \frac{1}{2} $ ist (natürlich nur wenn man davon ausgeht, dass eine Münze bei dieser Annahme nicht auf der Kante landen darf).
</div>

In [2]:
simple_df

,Text,Kategorie
1,Ein großartiges Spiel,Sport
2,Das Urteil ist beschlossen,Kein Sport
3,Ein sehr faires Match,Sport
4,Ein faires aber langweiliges Spiel,Sport
5,Es ist ein knappes Urteil,Kein Sport
6,Ein sehr knappes Spiel,?


Nun berechnen wir nacheinander die Wahrscheinlichkeit $ P $ der einzelnen Wörter, angefangen mit $ P(Ein|Sport) $. Dafür zählen wir zunächst, wie oft das Wort "Ein" in Texten der Kategorie "Sport" vorkommt (3 Mal). Das teilen wir durch die Gesamtanzahl der Wörter aus den Texten der Kategorie "Sport" (12 Wörter). Wir erhalten also $ P(Ein|Sport) = \frac{3}{12}$.<br>
Diese Berechnung führen wir nun für jedes der Wörter des Satzes "Ein sehr knappes Spiel" durch und erhalten folgende Ergebnisse:<br>

$ P(sehr|Sport) = \frac{1}{12}$<br>
$ P(knappes|Sport) = \frac{0}{12} = 0 $<br>
$ P(Spiel|Sport) = \frac{2}{12}$<br>

Wir haben nun jedoch ein Problem: Das Wort "knappes" kommt in keinem der Texte aus der Kategorie "Sport" vor und die Wahrscheinlichkeit ist somit $0$. Würden wir nun aber die Berechnung $ P(Ein|Sport) \cdot P(sehr|Sport) \cdot P(knappes|Sport) \cdot P(Spiel|Sport) $ ausführen, würde der <u>gesamte</u> Ausdruck $0$ werden, da $ P(Ein|Sport) \cdot P(sehr|Sport) \cdot 0 \cdot P(Spiel|Sport) = 0$.<br>
Wir müssen dieses Problem umgehen, da wir ansonsten nur Sätze klassifizieren könnten, bei denen alle Wörter in jeder Kategorie vorkommen. Wir wenden das sogenannte <b>Laplace smoothing</b> an. Dies bedeutet, dass wir zu jedem gezählten Wort 1 addieren, damit das Produkt nie 0 werden kann. Damit wäre es jedoch möglich, dass wir eine Wahrscheinlichkeit > 1 erhielten, wenn alle Texte einer Kategorie nur aus einem Wort bestünden. Würden alle Texte der Kategorie "Sport" also aus dem einzelnen Wort "Spiel" bestehen, wäre $ P(Spiel|Sport) = \frac{3+1}{3} = \frac{4}{3} > 1$. Dieser Fall ist sehr unwahrscheinlich, trotzdem addieren wir im Divisor die Anzahl aller möglichen Wörter, nicht nur die der Kategorie "Sport", sondern auch der Kategorie "Kein Sport". Dieses Vokabular besteht in unserem Fall aus 14 Wörtern.<br>
$ P(Ein|Sport) $ mit Laplace Smoothing wäre also $ P(Ein|Sport)  = \frac{3+1}{12+14} = \frac{4}{26} $.

In [3]:
smaller_simple_df = simple_df[:-1]

results = set()
smaller_simple_df['Text'].str.lower().str.split().apply(results.update)
print("Vokabular: " + str(results) +"\n")
print("Größe des Vokabulars: " + str(len(results)))

Vokabular: {'ist', 'ein', 'aber', 'match', 'es', 'langweiliges', 'knappes', 'spiel', 'sehr', 'beschlossen', 'urteil', 'das', 'faires', 'großartiges'}

Größe des Vokabulars: 14


Nun berechnen wir für jedes Wort des Satzes "Ein sehr knappes Spiel" die Wahrscheinlichkeit für beide Kategorien.

<table style="width:75%" align="left">
  <tr>
    <th>Wort</th>
    <th>$ P(Wort|Sport) $</th>
    <th>$ P(Wort|Kein\ Sport) $</th>
  </tr>
  <tr>
    <td>Ein</td>
    <td>$ \frac{3+1}{12+14} = \frac{4}{26}$</td>
    <td>$ \frac{1+1}{9+14} = \frac{2}{23}$</td>
  </tr>
  <tr>
    <td>sehr</td>
    <td>$ \frac{1+1}{12+14} = \frac{2}{26}$</td>
    <td>$ \frac{0+1}{9+14} = \frac{1}{23}$</td>
  </tr>
    <tr>
    <td>knappes</td>
    <td>$ \frac{0+1}{12+14} = \frac{1}{26}$</td>
    <td>$ \frac{1+1}{9+14} = \frac{2}{23}$</td>
  </tr>
    <tr>
    <td>Spiel</td>
    <td>$ \frac{2+1}{12+14} = \frac{3}{26} $</td>
    <td>$ \frac{0+1}{9+14} = \frac{1}{23}$</td>
  </tr>
</table><br>

Zuletzt multiplizieren wir die Wahrscheinlichkeiten für jede Kategorie:<br>

$  P(Ein|Sport) \cdot P(sehr|Sport) \cdot P(knappes|Sport) \cdot P(Spiel|Sport) \cdot P(Sport) 
\\ = \frac{4}{26} \cdot \frac{2}{26} \cdot \frac{1}{26} \cdot \frac{3}{26} \cdot \frac{3}{5} 
\\ = 0,000031512$ <br>

$  P(Ein|Kein\ Sport) \cdot P(sehr|Kein\ Sport) \cdot P(knappes|Kein\ Sport) \cdot P(Spiel|Kein\ Sport) \cdot P(Kein\ Sport) 
\\ = \frac{2}{23} \cdot \frac{1}{23} \cdot \frac{2}{23} \cdot \frac{1}{23} \cdot \frac{2}{5} 
\\ = 0,000005718$ <br>

Da 0,000031512 > 0,000005718, ist die Wahrscheinlichkeit, dass der Satz "Ein sehr knappes Spiel" der Kategorie "Sport" angehört, höher.

## 3.4. Naive Bayes mit Scikit learn <a class="anchor" id="3-4"/>

Im vorherigen Beispiel hatten wir einen Datensatz, der aus nur fünf sehr kurzen Texten bestand und nur zwei Kategorien hatte. Damit war es sehr einfach, das Naive Bayes Klassifikationsverfahren händisch zu berechnen. Die meisten Datensätze sind jedoch um einiges größer. Dort ist es von Vorteil, das Naive Bayes Verfahren in Python (oder einer anderen Programmiersprache) zu implementieren oder auf die Implementierung der Software-Bibliothek <b>Scikit learn</b> zurückzugreifen. Im nächsten Kapitel werden wir uns einem größeren Datensatz widmen, in diesem Abschnitt greifen wir noch ein letztes Mal auf unserer kleinen Datensatz aus dem vorherigen Abschnitt zurück, da unser Fokus auf der Implementierung von Naive Bayes in Scikit learn liegen soll.

In [4]:
import pandas as pd

news = {1 : ["Ein großartiges Spiel", "Sport"], 
        2 : ["Das Urteil ist beschlossen", "Kein Sport"],
        3 : ["Ein sehr faires Match", "Sport"],
        4 : ["Ein faires aber langweiliges Spiel", "Sport"],
        5 : ["Es ist ein knappes Urteil", "Kein Sport"],
        6 : ["Ein sehr knappes Spiel", "?"]}
corpus = pd.DataFrame.from_dict(news, orient="index", columns=["Text", "Kategorie"])

corpus = corpus[:-1]
corpus

,Text,Kategorie
1,Ein großartiges Spiel,Sport
2,Das Urteil ist beschlossen,Kein Sport
3,Ein sehr faires Match,Sport
4,Ein faires aber langweiliges Spiel,Sport
5,Es ist ein knappes Urteil,Kein Sport


Hier sehen wir zunächst den Datensatz aus dem vorherigen Beispiel, den wir für die ersten Schritte in Scikit learn noch einmal betrachten und benutzen werden. Den Satz "Ein sehr knappes Spiel", den wir klassifizieren wollen, wurde zunächst aus dem Datensatz entfernt. Zudem haben wir unseren Datensatz "corpus" genannt.

#### Umwandlung der Kategorien in eine numerische Repräsentation

Die erste Aufgabe, die wir angehen müssen, um unseren Datensatz mit dem Naive Bayes Klassifizierungsverfahren nutzen zu können, ist wie im vorherigen Beispiel den Datensatz so umzuwandeln, dass das Verfahren mit ihm arbeiten kann, es also in eine <i>numerische Repräsentation</i> überführen.<br>
Das Einfachste ist es, die Kategorien in Zahlen umzuwandeln, damit diese als Liste dem Naive Bayes Klassifizierungsverfahren übergeben werden kann. Dazu wandeln wir zunächst die Kategorien in eine Liste um.

In [5]:
categories = list(corpus["Kategorie"])
categories

['Sport', 'Kein Sport', 'Sport', 'Sport', 'Kein Sport']

Nun wollen wir diese Kategorien in Zahlen umwandeln. Dabei erhält jede Kategorie eine Zahl zwischen $0$ und $n-1$, wobei $n$ die Anzahl aller Kategorien ist. Da hier nur zwei Kategorien vorliegen, sind die Zahlen $0$ und $1$. Eine Umwandlung können wir nun zunächst händisch durchführen. "Sport" wird zu einer $1$, "Kein Sport" zu einer $0$.

In [6]:
[1, 0, 1, 1, 0]

[1, 0, 1, 1, 0]

Dieser Vorgang nennt sich <b>Encoding</b> und kann von Scikit learn mithilfe der Klasse `LabelEncoder` automatisch durchgeführt werden. Dabei können wir auch direkt die Spalte unseres Datensatzes mit den Kategorien angeben.

In [7]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

labels = encoder.fit_transform(corpus["Kategorie"])
labels

array([1, 0, 1, 1, 0])

<div class="alert alert-info">
<b>Exkurs:</b> fit_transform <br>

Die Methode `fit_transform` ist eine immer wiederkehrende Methode von Scikit learn und besteht eigentlich aus den Komponenten `fit` und `transform`.<br>
Beim Schritt `fit` wird das Modell an die bereitgestellten Daten angepasst. Das Modell "lernt" hier aus den Daten. Bei Textdaten wird hier meistens ein Vokabular erlernt.<br>
Beim Schritt `transform ` werden die Daten entsprechend dem angepassten Modell transformiert und wir erhalten eine Ausgabe. Bei Textdaten werden die Daten anhand des Vokabulars in Vektoren transformiert.<br>
`fit_transform` macht beides zusammen und spart meistens ein paar Zeilen Code. In manchen Fällen ist es zwar hilfreich, die Schritte `fit` und `transform` einzeln auszuführen, in den meisten Fällen ist `fit_transform` aber praktikabler. 
</div>

#### Umwandlung des Textes in eine numerische Repräsentation

Neben den Labels müssen wir nun aber auch den Text in eine numerische Repräsentation überführen. Der `LabelEncoder` hilft uns hier nicht weiter, da er die Sätze als Ganzes transformiert und sie nicht nach Wörtern unterteilt.

In [8]:
wrong_text_transformation = LabelEncoder().fit_transform(corpus["Text"])
wrong_text_transformation

array([2, 0, 3, 1, 4])

Wie auch beim Beispiel aus dem vorherigen Abschnitt verwenden wir als numerische Repräsentation der Texte das <b>Bag-of-Words</b> Modell. Diese wird in Scikit learn in Form des `CountVectorizer` zur Verfügung gestellt. Ein Vectorizer <b>vektorisiert</b> die Daten, wobei Vektorisierung der allgemeine Vorgang ist, eine Sammlung von Textdokumenten in numerische Merkmalsvektoren umzuwandeln. Auch hier können wir `fit_transform` anwenden, wobei beim `fit` Schritt das Vokabular gelernt wird (auf Basis der Texte) und im `transform` Schritt die Texte anhand des Vokabulars in Vektoren transformiert werden.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(corpus["Text"])
print(vector)
vector

  (0, 12)	1
  (0, 6)	1
  (0, 3)	1
  (1, 1)	1
  (1, 7)	1
  (1, 13)	1
  (1, 2)	1
  (2, 10)	1
  (2, 5)	1
  (2, 11)	1
  (2, 3)	1
  (3, 9)	1
  (3, 0)	1
  (3, 5)	1
  (3, 12)	1
  (3, 3)	1
  (4, 8)	1
  (4, 4)	1
  (4, 7)	1
  (4, 13)	1
  (4, 3)	1


<5x14 sparse matrix of type '<class 'numpy.int64'>'
	with 21 stored elements in Compressed Sparse Row format>

Als Output erhalten wir eine Matrix, mit der wir zunächst nichts anfangen können. Schauen wir uns jedoch das Vokabular an, dass beim `fit` Schritt erstellt wurde, wird die Matrix etwas verständlicher.

In [10]:
vectorizer.vocabulary_

{'ein': 3,
 'großartiges': 6,
 'spiel': 12,
 'das': 2,
 'urteil': 13,
 'ist': 7,
 'beschlossen': 1,
 'sehr': 11,
 'faires': 5,
 'match': 10,
 'aber': 0,
 'langweiliges': 9,
 'es': 4,
 'knappes': 8}

Jedes Wort des Vokabulars hat eine eindeutige Zahl (ID) erhalten. So bestehen die Tupel in der Sparse Matrix aus der Nummer des Dokuments und der Wort-ID. (0, 12) steht für "spiel", (0, 6) steht für "großartiges" und (0,3) für "ein". Zusammen ergibt das "spiel großartiges ein". In der richtigen Reihenfolge und korrekten Groß- und Kleinschreibung ergibt das den Satz "Ein großartiges Spiel", den ersten Text unseres Datensatzes. Es ist eine Eigenschaft des Bag-of-Words Modells, dass die Reihenfolge der Wörter ignoriert wird.<br>
Die ausgegebene Matrix können wir noch etwas anders wiedergeben, und zwar in einer Form, die wirklich sparse ist. Jeder Eintrag der Matrix, also jede Zeile, repräsentiert einen unserer Texte. Die Spalten repräsentieren die einzelnen Wörter. An jeder Stelle, an der eine "1" steht, kommt das entsprechende Wort im Text vor. In der ersten Zeile der Matrix sind an dritter (wir fangen bei 0 an zu zählen), sechster und zwölfert Stelle Einsen. Gucken wir im Vokablur nach, erhalten wir den Satz "Ein großartiges Spiel".

In [11]:
vector.toarray()

array([[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1]], dtype=int64)

<div class="alert alert-info">
<b>Exkurs:</b> Sparse und Dense Matrizen <br>
    
<b>Sparse</b> und <b>Dense</b> Matrizen sind zwei Varianten von Matrizen. Der Unterschied zwischen den beiden Varianten ist, dass Sparse Matrizen viele Nullen enthalten und Dense Matrizen nicht. Sparse Matrizen sind beim Machine Learning oft das Ergebnis von Vektorisierungsverfahren wie <b>CountVectorizing</b> oder <b>TfidfVectorizing</b> oder das Ergebnis von <b>One-hot encoding</b>. Aufgrund der vielen Nullen nehmen sie viel Speicherplatz in Anspruch und verlangsamen Machine Learning Algorithmen, da sie beispielsweise bei Scikit learn in den RAM-Speicher geladen werden. Sie werden daher oft komprimiert und in Dense Matrizen umgewandelt. Für weitere Informationen zu Sparse und Dense Matrizen siehe den <a href="https://dziganto.github.io/Sparse-Matrices-For-Efficient-Machine-Learning/">Blog-Eintrag</a> von David Ziganto.

</div>

Nun haben wir unsere Daten in die richtige numerische Form gebracht und können sie unserem Naive Bayes Klassifizierungsverfahren übergeben. Wir nehmen hier das <b>Multinomial Naive Bayes</b> Klassifizierungsverfahren. Anders als bei dem Vectorizer gibt es hier keine `fit_transform` Methode. Stattdessen passen wir nur das Klassifizierungsverfahren mit `fit` an unsere vektorisierten Daten (`vector`) und unsere Kategorien (`labels`) an.

In [12]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()

mnb = classifier.fit(vector, labels)

Nun wollen wir aber unser Klassifizierungsverfahren voraussagen lassen, ob der Satz "Ein sehr knappes Spiel" zur Kategorie "Sport" (1) oder zur Kategorie "Kein Sport" (0) gehört. Dafür müssen wir den Satz gemäß unseres Vectorizers in eine Array überführen.

In [13]:
vector2 = vectorizer.transform(["Ein sehr knappes Spiel"])

vector2.toarray()

array([[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]])

Erst dann können wir mithilfe der `predict` Methode die Kategorie des Satzes voraussagen.

In [14]:
prediction = mnb.predict(vector2)
prediction

array([1])

Wenn wir lieber als Output die unkodierte Repräsentation der vorausgesagten Kategorie bevorzugen (was vor allem bei sehr vielen Kategorien hilfreich ist), können wir mithilfe unseres Encoders und der Methode `inverse_transform` die numerische Kategorierepräsentation invers transformieren.

In [15]:
original_predicted_category = encoder.inverse_transform(prediction)
str(original_predicted_category)

"['Sport']"

Unser Klassifizierungsverfahren sagt für den Satz "Ein sehr knappes Spiel" die Kategorie "Sport" voraus, wie auch bei unserer eigenen Berechnung im vorherigen Abschnitt.<br>

Hier nochmal eine gesamte Repräsentation des dafür benötigten Codes:

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Kategorien in numerische Repräsentation transformieren
encoder = LabelEncoder()
labels = encoder.fit_transform(corpus["Kategorie"])

# Text in numerische Repräsentation transformieren
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(corpus["Text"])

# Multinomial Naive Bayes
classifier = MultinomialNB()
mnb = classifier.fit(vector, labels)

# Unser Testsatz
vector2 = vectorizer.transform(["Ein sehr knappes Spiel"])

# Die Voraussage
prediction = mnb.predict(vector2)
print(prediction)
original_predicted_category = encoder.inverse_transform(prediction)
print(str(original_predicted_category))

[1]
['Sport']


## 3.5 Mögliche Fehler <a class="anchor" id="3-5"/>

- Kein Encoding und Vectorizing durchgeführt → Es kommen Fehlermeldungen wie `could not convert string to float`. Der erste Schritt bei der Nutzung eines Scikit learn Textklassifizierungsverfahren ist immer, die Klassen zu encoden und die Texte zu vektorisieren.
- Klassen und Texte vertauscht → Klassen werden encodet, Texte werden vektorisiert. Bestehen die Klassen auch aus Strings, funktioniert das Klassifizierungsverfahren zwar, es macht jedoch keinen Sinn (hier wäre das Ergebnis `[0]` und `['Das Urteil ist beschlossen']`).